### 1. PDF 를 페이지 단위로 나누기
### 2. 1번 작업 수행 후 청킹 작업으로 

In [8]:

import fitz
import os, yaml, json
import sys, os
project_root = os.path.abspath(r"C:\PythonProject\ai_kt_agent") 
sys.path.append(project_root)
from src.common import GlobalSetting
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 청킹용 스플리터 (보통 자주 많이씀)
# 교체 될수도 있음
spliter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", " ", ""]
)


# 일단 pdf 기준으로 했습니다. 
def chunk_pdf(filepath:
               str, source_name: str,
               year: int = None,
                lang: str = "en"):
    

    docs = []
    # 파일을 오픈해봅니다.
    try:
        doc = fitz.open(filepath)
    except Exception as e:
        print(f"열기 실패 {filepath}: {e}")
        return docs

    # 각 페이지의 텍스트와 페이지 넘버수 추출 
    # 나중에 어떤  document 를 참고했는지 확인하기 위해 페이지 넘버 넣었어요
    for page_num, page in enumerate(doc, start=1):
        text = page.get_text()
        if not text: continue

    chunks = spliter.split_text(text)
    for i, chunk in enumerate(chunks, start=1):
        docs.append(
            {
                "text": chunk,
                "page": page_num,
                "chunk_id": f"{page_num}_{i}",
                "source": os.path.basename(filepath),
                "name": source_name,
                "year": year,
                "lang": lang,
            }
        )
    
    return docs

SOURCE_FILE = os.path.join(GlobalSetting.DATA_DIR, "sources.yaml")
OUTPUT_FILE = os.path.join(GlobalSetting.DATA_DIR, "all_chunk_list.jsonl")

print(SOURCE_FILE)
print(OUTPUT_FILE)
# /data/sources.yaml를 찾아서 config 객체화 시킴 
with open(SOURCE_FILE,"r",encoding="utf-8") as f:
    config = yaml.safe_load(f)

# config의 sources 를 꺼내와서 사용한다. 이 sources 는 pdf 경로를 뜻하며 동시에 메타데이터 담기용임 
all_docs = []
for src in config["sources"]:
    filepath = os.path.join(GlobalSetting.DATA_DIR, src["file"])  # 경로 생성

    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        continue

    # source 내용의 pdf 가 있다면 청킹의 메타데이터 넘기기
    if src["type"] == "pdf":
        docs = chunk_pdf(filepath, src["name"], src.get("year"), src.get("lang","en"))
        all_docs.extend(docs)
        print(f"Parsed {src['name']} -> {len(docs)} pages")

# 청킹에서 추출된 문서들을 세그먼트라고 해요. -> 세그먼트 추출해서 전체 저장 
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for d in all_docs:
        f.write(json.dumps(d,ensure_ascii=False) + "\n")



C:\PythonProject\ai_kt_agent\src\data\sources.yaml
C:\PythonProject\ai_kt_agent\src\data\all_chunk_list.jsonl
Parsed HiKorea Main -> 2 pages
